In [1]:
#improting all the required packages
from __future__ import print_function
import mechanicalsoup
from bs4 import BeautifulSoup
import urllib.response
import urllib.request
from urllib.request import urlopen
import csv
import os
from zipfile import ZipFile
from io import BytesIO
import warnings
warnings.filterwarnings('ignore')

In [2]:
#function to check is directory exists
def funCheckDir(filepath):
    directory = os.path.dirname(filepath) # defining directory path
    if not os.path.exists(directory): # checking if directory already exists
        os.makedirs(directory) # making a directory
        return False
    else :
        return True

In [3]:
#defining the file-directory
fileDir = os.path.dirname(os.path.realpath('__file__'))

In [4]:
#defining base ulrs
loginURL = 'https://www.lendingclub.com/account/gotoLogin.action'

In [5]:
#ask for login credentials
username = input("Lending-club username : ")
password = input("Lending-club password : ")

Lending-club username : spawargautam@gmail.com
Lending-club password : Pawar@121


In [6]:
#trying to loging to the lending-club
browser = mechanicalsoup.Browser()
login_page = browser.get(loginURL)
login_form = login_page.soup.find('form', {"id":"member-login"})
login_form.find("input", {"name" : "login_email"})["value"] = username
login_form.find("input", {"name" : "login_password"})["value"] = password
response = browser.submit(login_form, login_page.url)

In [7]:
if (response.url == 'https://www.lendingclub.com/account/summary.action'):
    print(username + " logged in successfully!")
    url = 'https://www.lendingclub.com/info/download-data.action'
    baseURL = 'https://resources.lendingclub.com/'

    #opening an url and creating a soup
    print("Opening an url and creating a soup..")
    html = browser.get(url)
    soup = BeautifulSoup(html.text,"html.parser")

    #working on Loan Stats Data
    print("Looking for loan-stats data..")
    #getting file extentions
    loanStatFileExtn = soup.find( 'div' , { 'id' : 'loanStatsFileNamesJS' }).text
    loanStatFileExtn = loanStatFileExtn.split('|')
    #getting file names
    loanStatNames = [str(x.text) for x in soup.find(id="loanStatsDropdown").find_all('option')]
    #combining names and files
    loanStats = list(zip(loanStatFileExtn, loanStatNames))
    print("Total number of loan-stats data files : " + str(len(loanStats)))


    #downloading Loan Stats Data
    print("Downloading loan-stats data files...")
    for (extn, name) in loanStats:
        path = os.path.join(fileDir, 'data/raw_data/loan_data/'+str(name)+'/')
        if funCheckDir(path):
            continue
        else:
            print("Downloading data file for " + str(name))
            with urlopen(baseURL + extn) as zipresp:
                with ZipFile(BytesIO(zipresp.read())) as zfile:
                    zfile.extractall(path)

    print("Loan data location - " + os.path.join(fileDir, 'data/raw_data/loan_data/'))
    
    #working on Declined-Loan Stats Data
    print("Looking for declined-loan-stats data..")
    #getting file extentions
    loanStatFileExtn = soup.find( 'div' , { 'id' : 'rejectedLoanStatsFileNamesJS' }).text
    loanStatFileExtn = loanStatFileExtn.split('|')
    #getting file names
    loanStatNames = [str(x.text) for x in soup.find(id="rejectStatsDropdown").find_all('option')]
    #combining names and files
    loanStats = list(zip(loanStatFileExtn, loanStatNames))
    print("Total number of declined-loan-stats data files : " + str(len(loanStats)))


    #downloading Declined-Loan Stats Data
    print("Downloading declined-loan-stats data files...")
    for (extn, name) in loanStats:
        path = os.path.join(fileDir, 'data/raw_data/declined_loan_data/'+str(name)+'/')
        if funCheckDir(path):
            continue
        else:
            print("Downloading data file for " + str(name))
            with urlopen(baseURL + extn) as zipresp:
                with ZipFile(BytesIO(zipresp.read())) as zfile:
                    zfile.extractall(path)
    print("Declined-Loan data location - " + os.path.join(fileDir, 'data/raw_data/declined_loan_data/'))
else:
    print("Either username or password or both are wrong..")

spawargautam@gmail.com logged in successfully!
Opening an url and creating a soup..
Looking for loan-stats data..
Total number of loan-stats data files : 8
Loan data location - D:\Masters\Spring2017\Python\Final\data/raw_data/loan_data/
Looking for declined-loan-stats data..
Total number of declined-loan-stats data files : 7
Declined-Loan data location - D:\Masters\Spring2017\Python\Final\data/raw_data/declined_loan_data/
